In [13]:
import pandas as pd
import altair as alt
import numpy as np

In [14]:
data = pd.read_csv("games.csv")
#dropping rows with missing values
data.dropna(inplace=True)

data.head(5)

,Unnamed: 0,Title,Release Date,Team,Rating,Times Listed,Number of Reviews,Genres,Summary,Reviews,Plays,Playing,Backlogs,Wishlist
0,0,Elden Ring,"Feb 25, 2022","['Bandai Namco Entertainment', 'FromSoftware']",4.5,3.9K,3.9K,"['Adventure', 'RPG']","Elden Ring is a fantasy, action and open world...","[""The first playthrough of elden ring is one o...",17K,3.8K,4.6K,4.8K
1,1,Hades,"Dec 10, 2019",['Supergiant Games'],4.3,2.9K,2.9K,"['Adventure', 'Brawler', 'Indie', 'RPG']",A rogue-lite hack and slash dungeon crawler in...,['convinced this is a roguelike for people who...,21K,3.2K,6.3K,3.6K
2,2,The Legend of Zelda: Breath of the Wild,"Mar 03, 2017","['Nintendo', 'Nintendo EPD Production Group No...",4.4,4.3K,4.3K,"['Adventure', 'RPG']",The Legend of Zelda: Breath of the Wild is the...,['This game is the game (that is not CS:GO) th...,30K,2.5K,5K,2.6K
3,3,Undertale,"Sep 15, 2015","['tobyfox', '8-4']",4.2,3.5K,3.5K,"['Adventure', 'Indie', 'RPG', 'Turn Based Stra...","A small child falls into the Underground, wher...",['soundtrack is tied for #1 with nier automata...,28K,679,4.9K,1.8K
4,4,Hollow Knight,"Feb 24, 2017",['Team Cherry'],4.4,3K,3K,"['Adventure', 'Indie', 'Platform']",A 2D metroidvania with an emphasis on close co...,"[""this games worldbuilding is incredible, with...",21K,2.4K,8.3K,2.3K


In [15]:
#dropping duplicated and unnecessary rows
data.drop_duplicates(inplace=True)
data = data.drop(["Unnamed: 0","Reviews","Summary","Genres","Playing"], axis=1)
data.head(1)

,Title,Release Date,Team,Rating,Times Listed,Number of Reviews,Plays,Backlogs,Wishlist
0,Elden Ring,"Feb 25, 2022","['Bandai Namco Entertainment', 'FromSoftware']",4.5,3.9K,3.9K,17K,4.6K,4.8K


In [16]:
#renaming inconvenient columns
data.rename(columns={"Release Date":"Release", "Times Listed":"Listed", "Number of Reviews":"Reviews"}, inplace=True)
data.head(1)

,Title,Release,Team,Rating,Listed,Reviews,Plays,Backlogs,Wishlist
0,Elden Ring,"Feb 25, 2022","['Bandai Namco Entertainment', 'FromSoftware']",4.5,3.9K,3.9K,17K,4.6K,4.8K


In [17]:
#replacing k in numbers
data["Listed"] = data["Listed"].replace({"K":"*1e3"}, regex=True).map(pd.eval).astype(int)
data["Reviews"] = data["Reviews"].replace({"K":"*1e3"}, regex=True).map(pd.eval).astype(int)
data["Plays"] = data["Plays"].replace({"K":"*1e3"}, regex=True).map(pd.eval).astype(int)
data["Backlogs"] = data["Backlogs"].replace({"K":"*1e3"}, regex=True).map(pd.eval).astype(int)
data["Wishlist"] = data["Wishlist"].replace({"K":"*1e3"}, regex=True).map(pd.eval).astype(int)
data.head(1)

,Title,Release,Team,Rating,Listed,Reviews,Plays,Backlogs,Wishlist
0,Elden Ring,"Feb 25, 2022","['Bandai Namco Entertainment', 'FromSoftware']",4.5,3900,3900,17000,4600,4800


In [18]:
from ast import literal_eval
#turn team into a list
data['Team'] = data['Team'].apply(literal_eval)
data.head(1)

,Title,Release,Team,Rating,Listed,Reviews,Plays,Backlogs,Wishlist
0,Elden Ring,"Feb 25, 2022","[Bandai Namco Entertainment, FromSoftware]",4.5,3900,3900,17000,4600,4800


In [19]:
#get all entires with ratings higher than 4
ratings = data.query('Rating>=4')
ratings.head(5)
#group by studio and count them to find the amount made by each
rating = ratings.explode('Team').groupby(['Team'])['Team'].count().reset_index(name="count")
rating['count'].value_counts()

1      114
2       52
3       44
4       16
6        8
7        6
5        4
9        4
10       4
11       3
21       3
8        3
12       1
28       1
18       1
16       1
15       1
40       1
103      1
13       1
17       1
22       1
37       1
Name: count, dtype: int64

In [20]:
ratings_chart = alt.Chart(rating, title='amount of studios making games above 4 stars by the amount of 4+ star games that studio made').mark_line().encode(
    x=alt.X('count:Q', scale = alt.Scale(domainMax=25, clamp=True), title="Amount of games"),
    y=alt.Y('count(count):Q', title="amount of studios")
)
ratings_chart

alt.Chart(...)

In [21]:
chart = alt.Chart(data, title="Average amount of people that played game by the rating of that game").mark_boxplot(extent='min-max').encode(
    x=alt.X('Rating:Q', scale = alt.Scale(zero=False)),
    y='mean(Plays):Q',
).properties(
    width=600,
    height=400
)
chart

alt.Chart(...)

In [22]:
# group by teams to find the total amount of people that played games by studio and it's average rating among those.
studios = data.explode('Team').groupby('Team', as_index=False).agg({'Plays':'sum', 'Rating':'mean'})
#data.explode('Team').groupby(['Team'])['Plays'].sum().reset_index(name="sum")
#studios['sum'].value_counts()
studios.head(5)

,Team,Plays,Rating
0,(Archive) Square Product Development Division 4,2000,4.000000
1,07th Expansion,5700,4.275000
2,0UP Games,1300,4.100000
3,11 bit studios,5127,3.266667
4,110 Industries,100,2.600000


In [23]:
studio_chart = alt.Chart(studios, title="Average amount of people that played a studios games by studios average game rating").transform_loess('Rating', 'Plays').mark_line().encode(
    x=alt.X('Rating:Q'),
    y=alt.Y('Plays:Q',scale = alt.Scale(domainMax=15000, clamp=True))
).properties(
    width=600,
    height=400
)
studio_chart

alt.Chart(...)